In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from scipy.sparse import csr_matrix      # sparse matrices
import itertools
%matplotlib inline

os.chdir('/Users/gcptien/Dataset/')

def quiz_ans(word):
    print("\033[1;31;48m",word)
wiki = pd.read_csv('people_wiki.csv')

In [3]:
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [5]:
word_count = load_sparse_csr('people_wiki_word_count.npz')


In [6]:
os.chdir('/Users/gcptien/Desktop/ML/Machine Learning-Clustering & Retrieval/Week 2/')
map_index_to_word = pd.read_json('people_wiki_map_index_to_word.json',typ='series')

In [7]:
word_count

<59071x547979 sparse matrix of type '<class 'numpy.int64'>'
	with 10379283 stored elements in Compressed Sparse Row format>

In [8]:
import string
def remove_punctuation(text):
    translator = str.maketrans(' ',' ', string.punctuation)
    return text.translate(translator)

In [9]:
wiki['text_clean'] = wiki['text'].apply(remove_punctuation)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
word_count_matrix = vectorizer.fit_transform(wiki['text_clean'])
word_count_matrix


<59071x548552 sparse matrix of type '<class 'numpy.int64'>'
	with 10379254 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(word_count)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [18]:
print(wiki[wiki['name'] == 'Barack Obama'])

distances , indices = model.kneighbors(word_count[35817], n_neighbors=10) # 1st arg: word count vector

                                              URI          name  \
35817  <http://dbpedia.org/resource/Barack_Obama>  Barack Obama   

                                                    text  \
35817  barack hussein obama ii brk husen bm born augu...   

                                              text_clean  
35817  barack hussein obama ii brk husen bm born augu...  


In [19]:
print("{0:^10} | {1:^30} | {2:^20}".format("Index","Name","Distance"))
print("-----------------------------------------------------------------------")
for i in range(indices.shape[1]):
    print("{0:^10} | {1:30} | {2:<20}".format(indices[0,i],wiki.iloc[indices[0,i]][1],distances[0,i]))


  Index    |              Name              |       Distance      
-----------------------------------------------------------------------
  35817    | Barack Obama                   | 0.0                 
  24478    | Joe Biden                      | 33.075670817082454  
  28447    | George W. Bush                 | 34.39476704383968   
  35357    | Lawrence Summers               | 36.15245496505044   
  14754    | Mitt Romney                    | 36.16628264005025   
  13229    | Francisco Barrio               | 36.3318042491699    
  31423    | Walter Mondale                 | 36.40054944640259   
  22745    | Wynn Normington Hugh-Jones     | 36.49657518178932   
  36364    | Don Bonker                     | 36.6333181680284    
   9210    | Andy Anstett                   | 36.959437225152655  


In [62]:
def unpack_dict(matrix, map_index_to_word):
    ##table = list(map_index_to_word.sort('index')['category'])
    # if you're not using SFrame, replace this line with
    table = sorted(map_index_to_word, key=map_index_to_word.get)
    
    
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} \
               for i in range(num_doc) ]

wiki['word_count'] = unpack_dict(word_count, map_index_to_word)

In [52]:
table = sorted(map_index_to_word, key=map_index_to_word.get)
matrix = word_count
data = matrix.data
indices = matrix.indices
indptr = matrix.indptr

In [69]:
map_index_to_word.describe()

count    547979.000000
mean     273989.000000
std      158188.055917
min           0.000000
25%      136994.500000
50%      273989.000000
75%      410983.500000
max      547978.000000
dtype: float64

In [24]:
def top_words(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    row = wiki[wiki['name'] == name]
    for i in row['word_count']:
        word_count_table = pd.DataFrame(i.items(), columns = ['word','count'])
    return word_count_table.sort('count', ascending=False)

In [28]:
name = 'Barack Obama'
row = wiki[wiki['name'] == name]


In [37]:
for i in row['word_count']:
    word_count_table = pd.DataFrame(i.items())


ValueError: DataFrame constructor not properly called!